<a href="https://colab.research.google.com/github/9637231757/data-analysis-utils/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Required imports
import gzip
import pandas as pd
from datetime import datetime
import re
from collections import Counter
import urllib.request
import os
import sys # Import sys for sys.exit()

# Part 1: Data Loading and Cleaning

# Download and load the dataset
def download_and_load_data():
    url = "ftp://ita.ee.lbl.gov/traces/calgary_access_log.gz"
    local_file = "calgary_access_log.gz"

    # Download with error handling
    try:
        if not os.path.exists(local_file):
            print(f"Downloading dataset from {url}...")
            urllib.request.urlretrieve(url, local_file)
            print("Dataset downloaded successfully.")
    except Exception as e:
        print(f"Error downloading dataset: {e}")
        return None

    # Read the compressed file with latin1 encoding to handle non-UTF-8 bytes
    try:
        print(f"Reading dataset from {local_file}...")
        with gzip.open(local_file, 'rt', encoding='latin1') as f:
            lines = f.readlines()
        print("Dataset read successfully.")
        return lines
    except FileNotFoundError:
        print(f"Error: Local file {local_file} not found after download attempt.")
        return None
    except Exception as e:
        print(f"Error reading dataset: {e}")
        return None

# Parse log entries
def parse_log_entry(line):
    # Apache Common Log Format regex
    # Added '?' after * in ([^"]*) to make it non-greedy for the request part
    # Changed request parsing slightly to be more robust
    pattern = r'(\S+) (\S+) (\S+) \[(.*?)\] "([^"]*)" (\d+) (\S+)'
    match = re.match(pattern, line)

    if not match:
        # print(f"Warning: Line did not match pattern: {line.strip()}") # Uncomment for debugging unparsed lines
        return None

    try:
        host, _, _, timestamp_str, request, http_code, bytes_transferred = match.groups()

        # Parse timestamp - handle potential timezone issues gracefully
        try:
            # Remove the timezone offset before parsing if it exists, or handle it
            # The original format string "%d/%b/%Y:%H:%M:%S %z" is correct if the timezone is present
            # If the timezone is missing or malformed, this will raise ValueError
            timestamp = datetime.strptime(timestamp_str, "%d/%b/%Y:%H:%M:%S %z")
        except ValueError:
             # Try parsing without timezone if the above fails
            try:
                timestamp = datetime.strptime(timestamp_str.split(' ')[0], "%d/%b/%Y:%H:%M:%S")
                # print(f"Warning: Timestamp parsed without timezone for line: {line.strip()}") # Uncomment for debugging
            except ValueError:
                # print(f"Warning: Invalid timestamp format: {timestamp_str} in line: {line.strip()}") # Uncomment for debugging
                return None # Return None for truly invalid timestamps

        # Extract filename from request
        try:
            request_parts = request.split()
            filename = request_parts[1] if len(request_parts) > 1 else ""
            # Handle cases like 'GET / HTTP/1.0' where filename is '/'
            if filename.startswith('/'):
                 filename = filename[1:] # Remove leading slash

            # Extract file extension
            # Ensure filename is not empty before splitting
            file_ext = filename.split('.')[-1].lower() if '.' in filename and filename else 'no_ext'
            # Handle cases like '/' which has no extension
            if file_ext == filename.lower() and '.' not in filename:
                 file_ext = 'no_ext'


        except IndexError:
            # print(f"Warning: Could not extract filename from request: {request} in line: {line.strip()}") # Uncomment for debugging
            filename = ""
            file_ext = 'no_ext'
        except Exception as e:
             # print(f"Warning: Unexpected error extracting filename: {e} from request: {request} in line: {line.strip()}") # Uncomment for debugging
             filename = ""
             file_ext = 'no_ext'


        # Handle bytes_transferred
        try:
            bytes_transferred = int(bytes_transferred) if bytes_transferred != '-' else 0
        except ValueError:
            # print(f"Warning: Invalid bytes value: {bytes_transferred} in line: {line.strip()}") # Uncomment for debugging
            bytes_transferred = 0 # Default to 0 if bytes are not a valid number

        return {
            'host': host,
            'timestamp': timestamp,
            'filename': filename,
            'file_ext': file_ext,
            'http_code': int(http_code),
            'bytes': bytes_transferred
        }
    except Exception as e:
        # Catch any other unexpected errors during parsing a matched line
        # print(f"Warning: Unexpected error parsing matched line: {e} for line: {line.strip()}") # Uncomment for debugging
        return None


# Load and clean data
def load_and_clean_data():
    lines = download_and_load_data()
    if not lines:
        print("No lines loaded from the dataset.")
        return None

    print("Parsing log entries...")
    # Parse all log entries and filter out None values
    # Use a generator expression and filter for efficiency
    parsed_entries_gen = (parse_log_entry(line) for line in lines)
    parsed_entries = [entry for entry in parsed_entries_gen if entry is not None]
    print(f"Successfully parsed {len(parsed_entries)} entries out of {len(lines)}.")

    # If no entries parsed successfully, return None
    if not parsed_entries:
        print("Warning: No log entries were successfully parsed after filtering.")
        return None

    print("Creating DataFrame...")
    try:
        df = pd.DataFrame(parsed_entries)
        print("DataFrame created successfully.")
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        return None

    # The following steps are generally good practice but might be less critical
    # now that None entries are filtered before DataFrame creation.
    # However, keeping them adds robustness.

    # Explicitly convert 'timestamp' to datetime, coercing errors.
    # This ensures the column is datetime-like even if parsing failed for some rows.
    # Note: With filtering None entries above, this might be redundant but kept for safety
    print("Converting timestamp column to datetime...")
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    print("Timestamp column conversion complete.")


    # Remove rows where the timestamp could not be parsed (are NaT)
    # This step is less likely to remove rows after filtering None entries
    initial_rows = len(df)
    df.dropna(subset=['timestamp'], inplace=True)
    rows_removed_ts_na = initial_rows - len(df)
    if rows_removed_ts_na > 0:
        print(f"Removed {rows_removed_ts_na} rows with invalid timestamps after DataFrame creation.")


    # Check if DataFrame is empty after cleaning
    if df.empty:
        print("DataFrame is empty after cleaning.")
        return None

    # Now the .dt accessor will work correctly on the 'timestamp' column
    # Add a check to ensure 'timestamp' column is indeed datetime before accessing .dt
    if pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        print("Extracting date and hour from timestamp...")
        df['date'] = df['timestamp'].dt.strftime('%d-%b-%Y')
        df['hour'] = df['timestamp'].dt.hour
        print("Date and hour columns added.")
    else:
        print("Warning: 'timestamp' column is not datetime type after cleaning. Date/Hour columns skipped.")
        # Optionally, return None or an empty DataFrame if datetime conversion is critical
        # return None


    return df

# Load the data
df = load_and_clean_data()

# Only proceed to analysis if the DataFrame was successfully loaded
if df is None or df.empty:
    print("Failed to load data or no data available after cleaning. Analysis skipped.")
    # Use sys.exit() for exiting in a script context, but exit() might be sufficient in Jupyter
    sys.exit("Data loading and cleaning failed.")
else:
    print(f"\nData loaded and cleaned. DataFrame shape: {df.shape}")
    print("DataFrame columns:", df.columns.tolist())
    # print("\nSample data:") # Uncomment to see sample data
    # display(df.head())

    # Part 2: Analysis Questions

    # Q1: Count of total log records
    def q1_total_records(df):
        return len(df)

    # Q2: Count of unique hosts
    def q2_unique_hosts(df):
        return df['host'].nunique()

    # Q3: Date-wise unique filename counts
    def q3_datewise_filenames(df):
        # Add a check to ensure 'date' column exists
        if 'date' in df.columns:
            print("Calculating date-wise unique filename counts...")
            result = df.groupby('date')['filename'].nunique().to_dict()
            print("Calculation complete.")
            return result
        else:
            print("Warning: 'date' column not found for Q3.")
            return {}


    # Q4: Number of 404 response codes
    def q4_count_404(df):
        print("Counting 404 responses...")
        count = len(df[df['http_code'] == 404])
        print("Count complete.")
        return count

    # Q5: Top 15 filenames with 404 responses
    def q5_top_15_404_filenames(df):
        print("Finding top 15 filenames with 404 responses...")
        df_404 = df[df['http_code'] == 404]
        if not df_404.empty:
             result = Counter(df_404['filename']).most_common(15)
        else:
             result = []
             print("No 404 responses found.")
        print("Calculation complete.")
        return result

    # Q6: Top 15 file extensions with 404 responses
    def q6_top_15_404_extensions(df):
        print("Finding top 15 file extensions with 404 responses...")
        df_404 = df[df['http_code'] == 404]
        if not df_404.empty:
            result = Counter(df_404['file_ext']).most_common(15)
        else:
             result = []
             print("No 404 responses found.")
        print("Calculation complete.")
        return result


    # Q7: Total bandwidth transferred per day for July 1995
    def q7_july_bandwidth(df):
        print("Calculating total bandwidth per day for July 1995...")
        # Add a check to ensure 'timestamp' is datetime type before filtering
        if pd.api.types.is_datetime64_any_dtype(df['timestamp']):
            # Ensure the timestamp includes the year for accurate filtering
            # Filter for entries in July 1995
            df_july_1995 = df[(df['timestamp'].dt.year == 1995) & (df['timestamp'].dt.month == 7)]

            # Add a check to ensure 'date' column exists in the filtered df and it's not empty
            if 'date' in df_july_1995.columns and not df_july_1995.empty:
                 result = df_july_1995.groupby('date')['bytes'].sum().to_dict()
                 print("Calculation complete.")
                 return result
            elif 'date' not in df_july_1995.columns:
                print("Warning: 'date' column not found for Q7 after filtering.")
                return {}
            else: # df_july_1995 is empty
                 print("No entries found for July 1995.")
                 return {}

        else:
            print("Warning: 'timestamp' column is not datetime type for Q7. Calculation skipped.")
            return {}


    # Q8: Hourly request distribution
    def q8_hourly_distribution(df):
        print("Calculating hourly request distribution...")
        # Add a check to ensure 'hour' column exists
        if 'hour' in df.columns:
            result = df['hour'].value_counts().sort_index().to_dict()
            print("Calculation complete.")
            return result
        else:
            print("Warning: 'hour' column not found for Q8.")
            return {}

    # Q9: Top 10 most requested filenames
    def q9_top_10_filenames(df):
        print("Finding top 10 most requested filenames...")
        result = Counter(df['filename']).most_common(10)
        print("Calculation complete.")
        return result

    # Q10: HTTP response code distribution
    def q10_http_code_distribution(df):
        print("Calculating HTTP response code distribution...")
        result = df['http_code'].value_counts().to_dict()
        print("Calculation complete.")
        return result

    # Execute all questions and display results
    print("\n--- Analysis Results ---")
    print("Q1 - Total log records:", q1_total_records(df))
    print("Q2 - Unique hosts:", q2_unique_hosts(df))
    print("Q3 - Date-wise unique filename counts:", q3_datewise_filenames(df))
    print("Q4 - Number of 404 responses:", q4_count_404(df))
    print("Q5 - Top 15 filenames with 404 responses:", q5_top_15_404_filenames(df))
    print("Q6 - Top 15 file extensions with 404 responses:", q6_top_15_404_extensions(df))
    print("Q7 - Total bandwidth per day (July 1995):", q7_july_bandwidth(df))
    print("Q8 - Hourly request distribution:", q8_hourly_distribution(df))
    print("Q9 - Top 10 most requested filenames:", q9_top_10_filenames(df))
    print("Q10 - HTTP response code distribution:", q10_http_code_distribution(df))

Reading dataset from calgary_access_log.gz...
Dataset read successfully.
Parsing log entries...
Successfully parsed 724837 entries out of 726739.
Creating DataFrame...
DataFrame created successfully.
Converting timestamp column to datetime...
Timestamp column conversion complete.
Removed 413054 rows with invalid timestamps after DataFrame creation.
Extracting date and hour from timestamp...
Date and hour columns added.

Data loaded and cleaned. DataFrame shape: (311783, 8)
DataFrame columns: ['host', 'timestamp', 'filename', 'file_ext', 'http_code', 'bytes', 'date', 'hour']

--- Analysis Results ---
Q1 - Total log records: 311783
Q2 - Unique hosts: 2
Calculating date-wise unique filename counts...
Calculation complete.
Q3 - Date-wise unique filename counts: {'01-Aug-1995': 684, '01-Jul-1995': 388, '01-Jun-1995': 591, '01-May-1995': 467, '01-Sep-1995': 328, '02-Apr-1995': 438, '02-Aug-1995': 857, '02-Jul-1995': 400, '02-Jun-1995': 515, '02-May-1995': 702, '02-Sep-1995': 286, '03-Apr-199